# Clustering

The goal of this program is to visualize and cluster the data.

## Input
* Importing the libaries that are necessary:

In [1]:
import numpy as np

* Importing the necessary files:
 * Read files
 * Convert read files to data
 * Save data in dataframes

In [2]:
def lib(FileName):
    def openFile(FileName):
        read_data = open(f'{FileName}')
        data = read_data.read()
        data = data.splitlines()
        read_data.close()
        
        return data
        
    lib = {}
    
    for lines in openFile(FileName):
        line = lines.split()
        lib[line[0]] = line[1:]
            
    return lib

lib_data = lib('Data\Voorbeeld_clusterdata.txt')
lib_results = lib('Data\Voorbeeld_clusterresult.txt')

## Cluster methods
### K-means (KMCA):

uitleggen hoe het werkt

In [3]:
class KMCA:
    def __init__(self, k=6, seed=20, tol=0.001, max_iter=300):
        self.k = k + 1
        self.seed = seed
        self.tol = tol
        self.max_iter = max_iter
        
    def set_k(self, k):
        self.k = k
        
    def set_seed(self, seed):
        self.seed = seed
    
    def fit(self, data):
        self.input_data = data
        
        def normalize():
            normalized = dict()
            for i in self.input_data:
                norm = np.sqrt(np.sum(np.square(np.array(self.input_data[i]).astype(float))))
                normalized[i] = np.divide(np.array(self.input_data[i]).astype(float), norm)

            self.lib_norm = normalized
        
        
        def rand_label():
            labeld_data = dict()
            labels_id = dict()
            np.random.seed(self.seed)
            labels = np.random.randint(1,self.k,len(self.lib_norm))

            for i in range(len(self.lib_norm)):
                labeld_data[list(self.lib_norm.keys())[i]] = labels[i]

            for K in range(1,self.k):
                labels_id[K] = np.array([i for i,j in labeld_data.items() if j == K])
            
            return labels_id
            
        def centroid():
            centroids = dict()

            for k in range(1,self.k):
                centroids[k] = np.mean(np.array([self.lib_norm[index] for index in self.labeled_data[k]]), axis=0)

            return centroids
        
        def assign_cluster(unlabeld_data):
            lib_clust = {k:[] for k in range(1,self.k)}
            lijst = list()
            
            for cords in unlabeld_data:
                clust = np.argmin([np.sqrt(np.sum(np.square(np.subtract(self.centroids[k],unlabeld_data[cords])))) for k in range(1,self.k)])+1
                lib_clust[clust].append(cords)

            return lib_clust
        
        for iteration in range(self.max_iter):
            if iteration == 0:
                norm = normalize()
                self.labeled_data = rand_label()
                self.centroids = centroid()
                
            else:
                optimized = True
                
                prev_centroids = self.centroids
                self.labeled_data = assign_cluster(self.lib_norm)
                
                self.centroids = centroid()
                
                for k in range(1,self.k):
                    if np.sum(((prev_centroids[k]-self.centroids[k])/prev_centroids[k])*100.0) > self.tol:
                        optimized = False
                        
                if optimized:
                    break
                
    def get_labels(self):
        return self.labeled_data

In [4]:
oke = KMCA()
oke.fit(lib_data)
results = oke.get_labels()

In [5]:
for cluster in results:
    print(f'cluster {cluster}: heeft {len(results[cluster])} data punten in zich')

cluster 1: heeft 97 data punten in zich
cluster 2: heeft 32 data punten in zich
cluster 3: heeft 209 data punten in zich
cluster 4: heeft 4 data punten in zich
cluster 5: heeft 488 data punten in zich
cluster 6: heeft 182 data punten in zich


### Own cluster method: Grid based Cluster Algorithm (GCA)
VERDERE UITLEG MOET NOG KOMEN

In [6]:
class GCA:
    """
    THIS CLASS USES Dictionaries as input
    """
    
    def __init__(self, r=1, j=10, tresh=4):
        self.r = r
        self.j = j
        self.tresh = tresh
        
    def set_range(self, r):
        self.r = r
        
    def set_interval(self,j):
        self.j = j
    
    
    def fit(self, data):
        self.indexes = list(data.keys())

        def get_cordinates(data):
            cordinates = dict()

            for cor in range(len(data[self.indexes[0]])):
                x = list()
                for i in data:
                    x.append(float(data[i][cor]))

                cordinates[cor] = tuple(x)
            return cordinates

        def get_intervals(data,j):
            dic = dict()

            for i in data:
                minim = np.min(data[i])
                maxim = np.max(data[i])

                dic[i] = np.mgrid[minim:maxim:complex(0,j)]

            return dic

        def get_cells():
            cordinates = get_cordinates(data)
            intervals = get_intervals(cordinates,self.j)
            
            labels = list()
            for i in cordinates:
                x_inter_label = list()
                for j in range(1,len(intervals[i])):
                    vals = list((cordinates[i] >= intervals[i][j-1]) & (cordinates[i] < intervals[i][j]))
                    vals = np.where(vals==False, 0, vals)
                    vals = np.where(vals==True, j, vals)       

                    x_inter_label.append(vals)

                labels.append(x_inter_label)


            labelzz = list()
            for i in range(len(labels)):
                for k in range(1,len(labels[i])):
                    labels[i][0] = np.add(labels[i][0],labels[i][k])
                labelzz.append(labels[i][0])

            del labels

            self.lib_cells = dict()     

            for i in range(len(labelzz[0])):
                cell_id = list()
                for j in range(len(labelzz)):
                    cell_id.append(labelzz[j][i])

                self.lib_cells[self.indexes[i]] = cell_id
            
            
            return self.lib_cells
        return get_cells()
        
    def self_predict(self, cell_ids):
        """
        DIT STUK MOET NOG EFFICIENTER WORDEN GEMAAKT

        """
        def get_neigb_id(ID):
            neigb = list()
            for indx in cell_ids:
                distance = np.subtract(self.lib_cells[ID],self.lib_cells[indx],out=np.array([0 for i in range(8)]))
                if all([-self.r < i < self.r for i in distance]):
                    neigb.append(indx)

            return neigb

        new_indexes = self.indexes.copy()

        def make_groups(data_point):
            neigbors = get_neigb_id(data_point)
            new_neigbors = list()

            for cel in neigbors:
                new_cells = get_neigb_id(cel)

                for new_cel in new_cells:
                    if new_cel not in neigbors:
                        new_neigbors.append(new_cel)
                        print(new_cel)

                neigbors += new_neigbors

            for neigb in neigbors:
                new_indexes.remove(neigb)

            return neigbors

        options = list()

        for index in new_indexes:
            options.append(make_groups(index))


        self.labeled_data = dict()
        cluster = 0
        anti_cluster = -1

        for i in range(len(options)):
            if len(options[i]) > self.tresh:
                self.labeled_data[cluster] = options[i]
                cluster += 1
            else:
                self.labeled_data[anti_cluster] = options[i]
                anti_cluster -= 1

        return self.labeled_data
            
    def predict(self, data):
        """
        VERZIN CODE DIE KIJKT WELKE CLUSTER DMV VAN CEL-ID GATHERING HET DICHTSTE BIJ IS

        """
        

In [7]:
gca = GCA()
fitted_data = gca.fit(lib_data)
results = gca.self_predict(fitted_data)

In [8]:
for cluster in results:
    print(f'cluster {cluster}: heeft {len(results[cluster])} data punten in zich')

cluster 0: heeft 10 data punten in zich
cluster 1: heeft 20 data punten in zich
cluster 2: heeft 53 data punten in zich
cluster -1: heeft 4 data punten in zich
cluster 3: heeft 5 data punten in zich
cluster 4: heeft 7 data punten in zich
cluster 5: heeft 22 data punten in zich
cluster 6: heeft 10 data punten in zich
cluster -2: heeft 4 data punten in zich
cluster 7: heeft 8 data punten in zich
cluster -3: heeft 1 data punten in zich
cluster -4: heeft 1 data punten in zich
cluster 8: heeft 28 data punten in zich
cluster 9: heeft 19 data punten in zich
cluster 10: heeft 12 data punten in zich
cluster 11: heeft 33 data punten in zich
cluster 12: heeft 15 data punten in zich
cluster 13: heeft 7 data punten in zich
cluster 14: heeft 7 data punten in zich
cluster 15: heeft 17 data punten in zich
cluster -5: heeft 2 data punten in zich
cluster -6: heeft 1 data punten in zich
cluster 16: heeft 13 data punten in zich
cluster -7: heeft 2 data punten in zich
cluster 17: heeft 60 data punten in zi